<a href="https://colab.research.google.com/github/i1idan/schizophrenia-diagnosis-eeg-signals/blob/main/colab/train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Clone the Repository

In [8]:
!git clone https://github.com/i1idan/schizophrenia-diagnosis-eeg-signals.git

Cloning into 'schizophrenia-diagnosis-eeg-signals'...
remote: Enumerating objects: 60, done.
remote: Counting objects: 100% (60/60), done.
remote: Compressing objects: 100% (44/44), done.
remote: Total 60 (delta 22), reused 45 (delta 13), pack-reused 0
Unpacking objects: 100% (60/60), done.


In [9]:
import os
os.chdir('/content/schizophrenia-diagnosis-eeg-signals')
!git pull origin main

From https://github.com/i1idan/schizophrenia-diagnosis-eeg-signals
 * branch            main       -> FETCH_HEAD
Already up to date.


# Install Requirements

In [10]:
!pip install -r requirements.txt

# Get Data

In [11]:
!gdown --id 1jnWHWrArzJQIvny0cQkfPP42hEJAp_56
!mv DATA.mat /content/schizophrenia-diagnosis-eeg-signals/data/DATA.mat

Downloading...
From: https://drive.google.com/uc?id=1jnWHWrArzJQIvny0cQkfPP42hEJAp_56
To: /content/schizophrenia-diagnosis-eeg-signals/DATA.mat
100% 299M/299M [00:02<00:00, 117MB/s]


# Training Models

In [12]:
# Number of training in a sequence
multi_train = 10
epochs = 200
model_name = "FFTCustom"
data_path = "./data/DATA.mat"
checkpoints = "/content/drive/MyDrive/schizophrenia/checkpoints"
batch_size = 4
early_stopping = 100
reduce_lr = 50
seed = 1234

In [ ]:
csv_files = []
for n in range(multi_train):
    dir_name = f"{n}"
    seed += 1
    !python train.py --model-name $model_name --epochs $epochs --seed $seed --dir-name $dir_name --checkpoints $checkpoints --batch-size $batch_size
    print(f"-----------------------------train {n} is done! ----------------------------------")

[INFO] Model:FFTCustom is loaded ...
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 110, 110, 8)       80        
                                                                 
 conv2d_1 (Conv2D)           (None, 109, 109, 8)       264       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 54, 54, 8)        0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 54, 54, 8)         0         
                                                                 
 conv2d_2 (Conv2D)           (None, 53, 53, 16)        528       
                                                                 
 conv2d_3 (Conv2D)           (None, 52, 52, 16)        1040      
                   

# Get Metrics

In [ ]:
from utils.group_metrics import get_mean_std

csv_files = [os.path.join(checkpoints, model_name ,dir_name, "log.csv") for n in range(multi_train)]
metrics = get_mean_std(csv_files, 
                       arguments=("accuracy", "loss", "val_accuracy", "val_loss"),
                       operators=(max, min, max, min)
                      )
print(metrics)

[INFO] Extracting values from csv files...
[INFO] Getting the values of file ./checkpoints/FFTCustom/1/log.csv
[INFO] Getting the values of file ./checkpoints/FFTCustom/1/log.csv
{'accuracy': {'std': 0.0, 'mean': 0.6498}, 'loss': {'std': 0.0, 'mean': 0.5906}, 'val_accuracy': {'std': 0.0, 'mean': 0.662}, 'val_loss': {'std': 0.0, 'mean': 0.5724}}
